## Basis Swap Par Spreads

The goal of the project is to compute the par spreads of a pool of basis swaps, i.e. those contracts in which two floating libor rates, with different tenors, are exchanged. The par spread is that margin which has to be paid/received on top of the floating rate with the shorter tenor.

The input parameters are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
* 3 sets of forward libor rates and their fixing dates, one for the Libor 1M, one for the Libor 3M and one for the Libor 6M.
* The static data of the pool of swaps (nominal, maturity, tenor of the first leg, tenor of the second leg)

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for multi-curve evaluation.

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!

In [1]:
import pandas as pd
from finmarkets import DiscountCurve, OvernightIndexSwap, generate_swap_dates, ForwardRateCurve
from datetime import date

today = date(2019, 10, 31)

ois_quotes = pd.read_csv("ois_quotes.csv").to_dict('records')

df = pd.read_csv("euribor1M_proj1.csv")
euribor_1m_curve = ForwardRateCurve(pd.to_datetime(df['dates']).dt.date.to_list(), 
                                    df['rates'])

df = pd.read_csv("euribor3M_proj1.csv")
euribor_3m_curve = ForwardRateCurve(pd.to_datetime(df['dates']).dt.date.to_list(), 
                                    df['rates'])

df = pd.read_csv("euribor6M_proj1.csv")
euribor_6m_curve = ForwardRateCurve(pd.to_datetime(df['dates']).dt.date.to_list(), 
                                    df['rates'])

basis_swaps = pd.read_csv("basis_swaps_proj1.csv").to_dict('records')

First of all we need to determine the discount curve from the OIS market quotes using the bootstrap technique.

* generate set of OIS according to the market quotes
* create an objective function with a discount curve and the squared sum of NPVs
* check minimization results
* create the found `DiscountCurve`

In [2]:
from finmarkets import generate_swap_dates, OvernightIndexSwap
pillar_dates = [today]

swaps = []
for quote in ois_quotes:
    swap = OvernightIndexSwap(
           1e6, 
           generate_swap_dates(today, quote['maturity']),
           0.01 * quote['rate'])
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])
    
pillar_dates = sorted(pillar_dates)

In [3]:
from finmarkets import DiscountCurve
import numpy as np

def objective_function(x):
    x = np.insert(x, 0, 1)
    curve = DiscountCurve(today, pillar_dates, x)
    
    sum_sq = 0.0
    for swap in swaps:
        sum_sq += swap.npv(curve) ** 2
        
    return sum_sq

In [4]:
from scipy.optimize import minimize

x0 = [1.0 for i in range(len(pillar_dates)-1)]
bounds = [(0.01, 100.0) for i in range(len(x0))]

result = minimize(objective_function, x0, bounds=bounds)

In [5]:
print (result)
print (objective_function(x0))
print (objective_function(result.x))

      fun: 0.0007487918928515683
 hess_inv: <30x30 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.70231666e-05,  1.88951501e-05, -6.58035689e-05, -5.06573947e-06,
        1.49836739e-06, -9.36252373e-05,  4.93635295e-05, -1.09646558e-04,
        3.85117283e-06,  3.51473840e-05,  4.97705824e-05,  2.38684722e-05,
        6.35263106e-05,  8.05371391e-05,  6.32675983e-05,  6.89265590e-05,
       -7.26789285e-05,  3.35287785e-05,  5.86377623e-05, -6.27189053e-05,
        5.35635119e-05, -2.05503473e-05,  5.74030295e-05, -2.62412378e-05,
       -6.08652340e-05, -1.78349522e-05, -9.95253462e-05, -1.72675458e-06,
        8.59563284e-05, -1.01028159e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 248
      nit: 6
     njev: 8
   status: 0
  success: True
        x: array([0.99999911, 0.99999806, 0.99999706, 0.99999606, 0.99999497,
       0.99999388, 0.99999278, 0.99999163, 0.99999025, 0.9999889 ,
       0.99998743, 0.99998597, 0.99998066, 0.99997477,

In [7]:
x = np.insert(result.x, 0, 1)
ois_curve = DiscountCurve(today, pillar_dates, x)

In [8]:
from finmarkets import ForwardRateCurve

euribor_curves = {}
euribor_curves[1] = euribor_1m_curve
euribor_curves[3] = euribor_3m_curve
euribor_curves[6] = euribor_6m_curve

Create a BasisSwap class that has a method to compute the fair margin as follows:

$$S = \frac {S_{t2} - S_{t1}}{\text{annuity}_{t1}} $$

In [9]:
bs_mat = [bsw['maturity'] for bsw in basis_swaps]
bs_ft = [bsw['first_tenor'] for bsw in basis_swaps]
bs_st = [bsw['second_tenor'] for bsw in basis_swaps]

In [10]:
from datetime import date
from dateutil.relativedelta import relativedelta
from finmarkets import generate_swap_dates

class BasisRateSwap:
    def __init__(self, start_date, notional, 
                 maturity, first_tenor, second_tenor):
    
        self.start_date = today
        self.notional = notional
        self.maturity = maturity
        self.tenor1 = first_tenor
        self.tenor2 = second_tenor
    
        self.tenor1_payment_dates = \
            generate_swap_dates(self.start_date, 
                                self.maturity, 
                                self.tenor1)
        self.tenor2_payment_dates = \
            generate_swap_dates(self.start_date, 
                                self.maturity, 
                                self.tenor2)

    def swap_rate(self, pd, dc, euribor, corr=0):
        sp = 0 
        for i in range(1, len(pd)):
            F = euribor.forward_rate(pd[i-1]) + corr
            D = dc.df(pd[i])
            tau = (pd[i] - pd[i-1]).days / 360
            sp += F * D * tau
        return sp
        
    def annuity(self, payment_dates, dc):
        annuity_ft = 0
        for j in range(1, len(payment_dates)):
            D = dc.df(payment_dates[j])
            tau = (payment_dates[j] - payment_dates[j-1]).days/360
            annuity_ft += D * tau
        return annuity_ft
        
    def fair_margin(self, discount_curve, euribor_1t, euribor_2t):
        s2 = self.swap_rate(self.tenor2_payment_dates, discount_curve, euribor_2t)
        s1 = self.swap_rate(self.tenor1_payment_dates, discount_curve, euribor_1t) 
        delta_sp = s2 - s1
        
        annuity_ft = self.annuity(self.tenor1_payment_dates, discount_curve)
        return (delta_sp / annuity_ft)
    
    def fair_npv(self, discount_curve, euribor_1t, euribor_2t):
        par_spread = self.fair_margin(discount_curve, euribor_1t, euribor_2t)
        npv2 = self.swap_rate(self.tenor2_payment_dates, discount_curve, euribor_2t)
        npv1 = self.swap_rate(self.tenor1_payment_dates, discount_curve, euribor_1t, par_spread)
        
        return (npv1 - npv2)* self.notional
    
    def fair_npv_with_par(self, par_spread, discount_curve, euribor_1t, euribor_2t):
        npv2 = self.swap_rate(self.tenor2_payment_dates, discount_curve, euribor_2t)
        npv1 = self.swap_rate(self.tenor1_payment_dates, discount_curve, euribor_1t, par_spread)
        
        return (npv1 - npv2)* self.notional        
    
    def npv(self, discount_curve, euribor_1t, euribor_2t):
        par_spread = self.fair_margin(discount_curve, euribor_1t, euribor_2t)
        npv2 = self.swap_rate(self.tenor2_payment_dates, discount_curve, euribor_2t)
        npv1 = self.swap_rate(self.tenor1_payment_dates, discount_curve, euribor_1t)
        
        return (npv1 - npv2)* self.notional

In [11]:
par_spreads = []

for i in range(len(bs_mat)):
    par_spreads.append(BasisRateSwap(today, 1000000, bs_mat[i],
                       bs_ft[i], bs_st[i]).fair_margin(ois_curve, 
                                                       euribor_curves[bs_ft[i]],
                                                       euribor_curves[bs_st[i]]))

print ("Par spreads: " + str(["{:.5f}".format(s) for s in par_spreads]))

Par spreads: ['0.00030', '0.00129', '0.00159', '0.00008', '0.00127', '0.00135', '-0.00005', '0.00122', '0.00117', '-0.00008', '0.00117', '0.00110', '-0.00009', '0.00113', '0.00104']


In [12]:
bs = BasisRateSwap(today, 100, bs_mat[2], bs_ft[2], bs_st[2])

In [13]:
bs.fair_npv(ois_curve, euribor_curves[bs_ft[2]], euribor_curves[bs_st[2]])


8.673617379884035e-17

In [14]:
bs.npv(ois_curve, euribor_curves[bs_ft[2]], euribor_curves[bs_st[2]])

-0.16115462465651056

### Alternative solution with brentq

In [15]:
from scipy.optimize import brentq

for i in range(len(bs_mat)):
    bs = BasisRateSwap(today, 1, bs_mat[i], bs_ft[i], bs_st[i])
    ps = brentq(bs.fair_npv_with_par, -1, 1, args=(ois_curve, euribor_curves[bs_ft[i]], euribor_curves[bs_st[i]]))
    print (ps)

0.00029858812301064575
0.0012865520053284563
0.0015851385661118655
8.149339056606753e-05
0.0012696016593209958
0.0013510928080062712
-4.913885113055905e-05
0.0012179176851027096
0.0011687710340888557
-7.812749623781201e-05
0.001174925259416204
0.001096785295187419
-9.280292864621842e-05
0.0011315867184754635
0.0010387680095655938
